In [1]:
# imports
import nltk
import pandas as pd
import numpy as np
import artm
import pymystem3

from nltk.corpus import brown
from nltk.stem import SnowballStemmer
from nltk.util import ngrams

mystem = pymystem3.Mystem()

In [43]:
# work environment setup - one-time actions
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\sbt-shekhovtsov-
[nltk_data]     rv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\sbt-shekhovtsov-
[nltk_data]     rv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [46]:
def clean_text(text):
    # remove numbers, single-char and stop-words
    words = nltk.word_tokenize(text)
    words = [w for w in words if w.isalpha() and len(w)>1] # and w not in stop_words]
    return ' '.join(words)

clean_text('мама мыла раму')

'мама мыла раму'

In [19]:
# MAKE STOP-WORDS DATASET
def make_stop_words_dataset():
    stop_words= nltk.corpus.stopwords.words('russian')
    names = pd.read_csv('names.csv')
    stop_words.extend(names['name'])
    #stop_words = list(map(lambda x: stemmer.stem(x), stop_words))
    #df = pd.read_csv('stop-words-russian-nltk.csv')
    df = pd.DataFrame(stop_words, columns=['stop-word'])
    #df['stop-word'] = df['stop-word'].apply(lambda x: clean_text(x))
    
    df.to_csv('stop-words-russian.csv', index=False, header=['stop-word'])

make_stop_words_dataset()

#stemmer = SnowballStemmer('russian')
# stop_word=" "
# for i in stop_words:
#         stop_word=  stop_word+" "+i
# print(stop_word)

In [18]:
#     stop_words= nltk.corpus.stopwords.words('russian')  
#     names = pd.read_csv('names.csv')
#     stop_words.extend(names['name'])
#df = pd.DataFrame(stop_words, columns=['stop-word'])
#df['stop-word'] = df['stop-word'].apply(lambda x: clean_text(x))


,stop-word
0,и
1,в
2,во
3,не
4,что
5,он
6,на
7,я
8,с
9,со


In [47]:
# DATA PREPARATION
# load raw data
def load_data():
    NROWS = None
    df = pd.read_csv( 'vk.csv', nrows=NROWS).fillna('')
    return df


# # stemming
# def stemming(df):
#     stemmer = SnowballStemmer('russian')
#     df['question_stem'] = df['question'].apply(lambda x: ' '.join([stemmer.stem(w) for w in str(x).split()]))
#     df['answer_stem'] = df['answer'].apply(lambda x: ' '.join([stemmer.stem(w) for w in str(x).split()]))
#     # df[['question_stem','answer_stem']]
#     # df.to_csv('vk-stemmed.csv', index=False, header=True)


def lemmatize(strings):
    s = mystem.lemmatize('||'.join(strings))
    s = ''.join(s)
    return s.split('||')


def data_preparation():
    df = load_data()
    # stemming(df)    
    
    df['question_clean'] = lemmatize(df['question'].values)
    df['answer_clean'] = lemmatize(df['answer'].values)

    df['question_clean'] = df['question_clean'].apply(lambda x: clean_text(x))
    df['answer_clean'] = df['answer_clean'].apply(lambda x: clean_text(x))
    df.to_csv('vk-cleaned.csv', index=False, columns=['question_clean','answer_clean'], header=['question','answer'])

# remove punctuation & stop-words
stop_words = list(pd.read_csv('stop-words-russian.csv')['stop-word'])
data_preparation()

In [41]:
#load_data()
# l = lemmatize(df['question'].values[:100])
# l2 = list(map(clean_text, l))
# l2


''

In [2]:
# SBER TOPICS
# stemmer = SnowballStemmer('russian')
mystem = pymystem3.Mystem()
df_topics = pd.read_csv('sber_topics_raw.csv', names=['topic'], sep='\n')
df_topics['topic'] = df_topics['topic'].apply(lambda x: clean_text(x))
#df_topics['topic'] = df_topics['topic'].apply(lambda x: clean_text(x))
#df_topics.head(200)
df_topics.to_csv('sber_topics.csv',index=False)

Installing mystem to C:\Users\sbt-shekhovtsov-rv/.local/bin\mystem.exe from http://download.cdn.yandex.net/mystem/mystem-3.1-win-64bit.zip


NameError: name 'clean_text' is not defined

In [22]:
df = pd.read_csv('vk-cleaned.csv')
df.head()

,question,answer
0,кредитн карт моментум расплачива зарубежн инте...,можете использова кредит карт cred momentum оп...
1,дебетн карт пришл сообщен неразрешен задолженн...,вопрос обрат служб помощ банк телефон специали...
2,вопрос поч приход смс пополнен карты рад подкл...,вопрос обрат служб помощ банк телефон специали...
3,зарплатн карт сб польз сбол кром перевод частн...,вопрос обрат служб помощ банк телефон специали...
4,пришл смс одобр предворительн кредит документ ...,потребительск кред выда соблюден след условий ...


In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidf = TfidfVectorizer()
# x = tfidf.fit_transform(df['question'].fillna(''))


In [30]:
# stop_words = list(pd.read_csv('stop-words-russian-nltk.csv')['stop-word'])

# from sklearn.feature_extraction.text import CountVectorizer
# countVectorizer = CountVectorizer(max_df=0.5, min_df=int(2),
#                                 max_features=10000,
#                                 stop_words=stop_words)
# x = countVectorizer.fit_transform(df['question'].fillna(''))

In [31]:
# x.shape

(40117, 10000)

In [32]:
# from sklearn.decomposition import LatentDirichletAllocation
# lda = LatentDirichletAllocation(n_components=250, max_iter=5,
#                                 learning_method='online',
#                                 learning_offset=50.,
#                                 random_state=0)
# lda.fit(x)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=250, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [ ]:
# def print_top_words(model, feature_names, n_top_words):
#     for topic_idx, topic in enumerate(model.components_):
#         message = "Topic #%d: " % topic_idx
#         message += " ".join([feature_names[i]
#                              for i in topic.argsort()[:-n_top_words - 1:-1]])
#         print(message)
#     print()
    
# feature_names = countVectorizer.get_feature_names()
# print_top_words(lda, feature_names, 20)

In [ ]:
# from sklearn.decomposition import NMF, LatentDirichletAllocation
# x = NMF(n_components=250, random_state=1, alpha=.1, l1_ratio=.5).fit(x)

In [44]:
def to_vw(df, columns_list, csv_name):
    df['vw_index'] = df.index.values
    df['vw_index'] = df['vw_index'].apply(lambda x: str(x))
    df['vw_text'] = 'd' + df['vw_index']
    for column_name in columns_list:
        df['vw_text'] += ' ' + df[column_name]
    df[['vw_text']].to_csv(csv_name, header=False, index=False)
    
df_topics = pd.read_csv('sber_topics.csv')
to_vw( df, ['question', 'answer'], 'vk.vw')
to_vw(df_topics, ['topic'], 'sber_topics.vw')

In [46]:
# create ARTM batches & dictionary - one-time action
batch_vectorizer = artm.BatchVectorizer(
    data_path = 'vk.vw',
    data_format='vowpal_wabbit',
    target_folder='artm')

dictionary = artm.Dictionary()
dictionary.gather(data_path='artm')
dictionary.save(dictionary_path='artm/dictionary')
dictionary.save_text(dictionary_path='artm/dictionary.txt')

In [47]:
batch_vectorizer = artm.BatchVectorizer(data_path='artm',
                                        data_format='batches')
dictionary = artm.Dictionary()
dictionary.load(dictionary_path='artm/dictionary.dict')
model = artm.ARTM(num_topics=200, dictionary=dictionary, cache_theta=True)
model.scores.add(artm.PerplexityScore(name='perplexity_score', dictionary=dictionary))
model.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score'))
model.scores.add(artm.SparsityThetaScore(name='sparsity_theta_score'))
model.scores.add(artm.TopTokensScore(name='top_tokens_score'))
model.num_document_passes = 5

In [48]:
model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=25)
#model.score_tracker['perplexity_score'].last_value

In [ ]:
print(model.score_tracker['perplexity_score'].value)      # .last_value
print()
print(model.score_tracker['sparsity_phi_score'].value)    # .last_value
print()
print(model.score_tracker['sparsity_theta_score'].value)  # .last_value

In [ ]:
for topic_name in model.topic_names:
    print(
        topic_name + ':',
        model.score_tracker['top_tokens_score'].last_tokens[topic_name]
    )

In [ ]:
model.score_tracker

In [ ]:
# def process_ngrams(s):
#     tokens = nltk.word_tokenize(s)
    

# df['question_ngram'] = dfp['question_stem'].apply(
#     lambda x: process_ngrams(x)
# )